In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import os
from sklearn.model_selection import train_test_split

In [ ]:
hengs_path = r'C:\Users\Kaggle\BengaliAI\hengcodes - NewAttemptseResnext - Cutmix'
sys.path.append(hengs_path)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from common  import *
# from dataset import *
from kaggle import *

# Stochastic Weight Averaging
# simply average the weights ... no bn post refinement
# https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/

def run_swa():

    # out_dir = \
    #     '/root/share/project/kaggle/2020/grapheme_classification/result/run1/resnet34-large-fold0'
    # snapshot = np.array([ 93000, 95000, 97000, 99000])
    # swa_name='swa_fold0_no_bn_model'

    out_dir = \
        r'C:\Users\Kaggle\BengaliAI\outputs_seresnext_with_mixup_cutmix_contin_2'
    snapshot = np.array([224999,376000,444000,449000 ]) #cyclic learning rate
    swa_name='swa_fold0_no_bn_model'


    ## --- net ---
    num_snapshot = len(snapshot)

    swa_state_dict = torch.load(out_dir +'/checkpoint/%08d_model.pth'%(snapshot[0]), map_location=lambda storage, loc: storage)
    for k,v in swa_state_dict.items():
        swa_state_dict[k] = torch.zeros_like(v)

    for iter in snapshot:
        checkpoint = out_dir +'/checkpoint/%08d_model.pth'%(iter)
        state_dict = torch.load(checkpoint, map_location=lambda storage, loc: storage)
        for k,v in state_dict.items():
            swa_state_dict[k] += v
        #---
        print(checkpoint)
    print('')

    #----
    for k,v in swa_state_dict.items():
        swa_state_dict[k] /= num_snapshot
    torch.save(swa_state_dict,out_dir +'/checkpoint/%s.pth'%swa_name)
    #torch.save(swa_state_dict,out_dir +'/checkpoint/swa_fold1_no_bn_model.pth')
    print("SWA success")

In [ ]:
run_swa()